In [1]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

In [2]:
import json
def output_format(action:str) -> str:
    #print("before format {}".format(action))
    action = action.split('```')[1].strip()
    action = json.loads(action)
    return action['action_input']

In [3]:
import sys
sys.path.append('/home/kit/kit/langchain')
from ChatGLM3 import ChatGLM3
model_path = "/data/tds/ChatGLM3/chatglm3-6b"
llm = ChatGLM3()
llm.load_model(model_name_or_path=model_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
llm("你好")

'\nAction: \n```\n{"action": "Final Answer", "action_input": "你好，请问有什么我可以帮助你的吗？"}\n```'

In [5]:
class CAMELAgent:
    def __init__(
        self,
        system_message: SystemMessage,#系统消息
        model:ChatGLM3,
    ) -> None:
        
        self.system_message = system_message
        self.model = model
        self.init_messages()
 
    def reset(self) -> None: #重置代理

        self.init_messages()
        return self.stored_messages
 
    def init_messages(self) -> None: #初始化消息，消息存储区只有系统消息
 
        self.stored_messages = [self.system_message]

    def update_messages(self, message: BaseMessage) -> List[BaseMessage]: #更新消息
 
        self.stored_messages.append(message) #将新消息加入消息存储区
        return self.stored_messages #返回消息存储区

    def step(
        self,
        input_message: HumanMessage,
    ) -> AIMessage:
 
        messages = self.update_messages(input_message) #更新消息
        prompt = ""
        for i in messages:
            prompt += i.content
        output_message = self.model(prompt) #利用大模型输出消息
        output_message = output_format(output_message)
        output_format_massage = HumanMessage(content=output_message)
        self.update_messages(output_format_massage) #更新消息
        return output_format_massage #返回输出
 
 
 

In [6]:
import os
#os.environ["OPENAI_API_KEY"] = "sk-Yeyc1tlCt5PLEz4zDdLgT3BlbkFJEpWcG32tNnDwkbVqQSv"
assistant_role_name = "Python Programmer"
user_role_name = "Stock Trader"
task = "Develop a trading bot for the stock market"
word_limit = 50 # 任务头脑风暴的字数限制

In [7]:
task_specifier_sys_msg = SystemMessage(content="You can make a task more specific.")#定义任务相关系统消息
task_specifier_prompt = (
"""Here is a task that {assistant_role_name} will help {user_role_name} to complete: {task}.
Please make it more specific. Be creative and imaginative.
Please reply with the specified task in {word_limit} words or less. Do not add anything else."""
)
#Here is a task that Python Programmer will help Stock Trader to complete: Develop a trading bot for the stock market.
#Please make it more specific. Be creative and imaginative.
#Please reply with the specified task in 50 words or less. Do not add anything else.
task_specifier_template = HumanMessagePromptTemplate.from_template(template=task_specifier_prompt)
 
task_specify_agent = CAMELAgent(task_specifier_sys_msg, llm)
task_specifier_msg = task_specifier_template.format_messages(assistant_role_name=assistant_role_name,
                                                             user_role_name=user_role_name,
                                                             task=task, word_limit=word_limit)[0]
specified_task_msg = task_specify_agent.step(task_specifier_msg)
print(f"Specified task: {specified_task_msg.content}")
specified_task = specified_task_msg.content

Specified task: Develop a trading bot that uses technical analysis to make buy and sell decisions for a specified list of stocks, with the goal of maximizing profits while minimizing risk. The bot should be able to process real-time market data, analyze it using various technical indicators, and execute trades based on its findings. Additionally, the bot should be able to continuously learn and adapt to changes in the market, and should be able to handle multiple stocks and multiple exchanges.


In [8]:
#创建AI助手和AI用户的启发式提示，用于角色扮演

assistant_inception_prompt = (
"""Never forget you are a {assistant_role_name} and I am a {user_role_name}. Never flip roles! Never instruct me!
We share a common interest in collaborating to successfully complete a task.
You must help me to complete the task.
Here is the task: {task}. Never forget our task!
I must instruct you based on your expertise and my needs to complete the task.
I must give you one instruction at a time.
You must write a specific solution that appropriately completes the requested instruction.
You must decline my instruction honestly if you cannot perform the instruction due to physical, moral, legal reasons or your capability and explain the reasons.
Do not add anything else other than your solution to my instruction.
You are never supposed to ask me any questions you only answer questions.
You are never supposed to reply with a flake solution. Explain your solutions.
Your solution must be declarative sentences and simple present tense.
Unless I say the task is completed, you should always start with:
Solution: <YOUR_SOLUTION>
<YOUR_SOLUTION> should be specific and provide preferable implementations and examples for task-solving.
Always end <YOUR_SOLUTION> with: Next request."""
)

user_inception_prompt = (
"""Never forget you are a {user_role_name} and I am a {assistant_role_name}. Never flip roles! You will always instruct me.
We share a common interest in collaborating to successfully complete a task.
I must help you to complete the task.
Here is the task: {task}. Never forget our task!
You must instruct me based on my expertise and your needs to complete the task ONLY in the following two ways:
1. Instruct with a necessary input:
Instruction: <YOUR_INSTRUCTION>
Input: <YOUR_INPUT>
2. Instruct without any input:
Instruction: <YOUR_INSTRUCTION>
Input: None
The "Instruction" describes a task or question. The paired "Input" provides further context or information for the requested "Instruction".
You must give me one instruction at a time.
I must write a response that appropriately completes the requested instruction.
I must decline your instruction honestly if I cannot perform the instruction due to physical, moral, legal reasons or my capability and explain the reasons.
You should instruct me not ask me questions.
Now you must start to instruct me using the two ways described above.
Do not add anything else other than your instruction and the optional corresponding input!
Keep giving me instructions and necessary inputs until you think the task is completed.
When the task is completed, you must only reply with a single word <CAMEL_TASK_DONE>.
Never say <CAMEL_TASK_DONE> unless my responses have solved your task."""
)

In [9]:
def get_sys_msgs(assistant_role_name: str, user_role_name: str, task: str):

    assistant_sys_template = SystemMessagePromptTemplate.from_template(template=assistant_inception_prompt)
    assistant_sys_msg = assistant_sys_template.format_messages(assistant_role_name=assistant_role_name, user_role_name=user_role_name, task=task)[0]
    
    user_sys_template = SystemMessagePromptTemplate.from_template(template=user_inception_prompt)
    user_sys_msg = user_sys_template.format_messages(assistant_role_name=assistant_role_name, user_role_name=user_role_name, task=task)[0]
    return assistant_sys_msg, user_sys_msg

In [10]:
#init
assistant_sys_msg, user_sys_msg = get_sys_msgs(assistant_role_name, user_role_name, specified_task)
assistant_agent = CAMELAgent(assistant_sys_msg, llm)
user_agent = CAMELAgent(user_sys_msg, llm)
 
# Reset agents
assistant_agent.reset()
user_agent.reset()
 
 
 
# Initialize chats 
assistant_msg = HumanMessage(
    content=(f"{user_sys_msg.content}. "
                "Now start to give me introductions one by one. "
                "Only reply with Instruction and Input."))
 
user_msg = HumanMessage(content=f"{assistant_sys_msg.content}")
user_msg = assistant_agent.step(user_msg)
print(user_msg.content)

Solution: I will start by importing the necessary libraries and modules for developing the trading bot, including pandas for data manipulation, numpy for numerical operations, and matplotlib for visualization. Next, I will create a function to load the stock data from a CSV file and format it into a pandas DataFrame. Then, I will implement the technical indicators and analysis logic in the bot, using various functions and methods to calculate relevant metrics and indices. After that, I will implement the trading strategy, which involves making buy and sell decisions based on the indicator values and risk management criteria. Finally, I will integrate the bot with the chosen exchanges and implement the real-time data processing and trade execution logic. Next request.


In [11]:
#print task message
print(f"Original task prompt:{task}")
print(f"Specified task prompt:{specified_task}")


chat_turn_limit, n = 30, 0
while n < chat_turn_limit:
    #记录轮数
    n += 1

    #AI user working
    user_ai_msg = user_agent.step(assistant_msg)
    user_msg = HumanMessage(content=user_ai_msg.content)
    print(f"AI User ({user_role_name}):{user_msg.content}")

    #AI assistant working
    assistant_ai_msg = assistant_agent.step(user_msg)
    assistant_msg = HumanMessage(content=assistant_ai_msg.content)
    print(f"AI Assistant ({assistant_role_name}):{assistant_msg.content}")

    #结束任务
    if "<CAMEL_TASK_DONE>" in user_msg.content:
        break
 
 
 

Original task prompt:Develop a trading bot for the stock market
Specified task prompt:Develop a trading bot that uses technical analysis to make buy and sell decisions for a specified list of stocks, with the goal of maximizing profits while minimizing risk. The bot should be able to process real-time market data, analyze it using various technical indicators, and execute trades based on its findings. Additionally, the bot should be able to continuously learn and adapt to changes in the market, and should be able to handle multiple stocks and multiple exchanges.
AI User (Stock Trader):Instruction: To start, I need you to provide a list of stocks that the trading bot will be making buy and sell decisions for.
Input: Can you provide a list of stocks to trade, such as Apple, Amazon, and Microsoft?
AI Assistant (Python Programmer):Solution: I will start by importing the necessary libraries and modules for developing the trading bot, including pandas for data manipulation, numpy for numeric

Input length of input_ids is 9472, but `max_length` is set to 8192. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


AI Assistant (Python Programmer):list_of_stocks


../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1125,0,0], thread: [0,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1125,0,0], thread: [1,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1125,0,0], thread: [2,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1125,0,0], thread: [3,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1125,0,0], thread: [4,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1125,0,0], thread: [5,0,0] Assertion 

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling cublasLtMatmul with transpose_mat1 1 transpose_mat2 0 m 4608 n 9472 k 4096 mat1_ld 4096 mat2_ld 4096 result_ld 4608 abcType 2 computeType 68 scaleType 0